# Exploring the QPTDat ontologies


In [1]:
import rdflib
import owlrl

#from rdflib import Graph, Literal, URIRef, BNode
#from rdflib.term import Identifier
#from rdflib.collection import Collection
#from rdflib.namespace import RDF, RDFS, SKOS, XSD, OWL
#import rdflib.plugins.sparql.update
#import owlrl.RDFSClosure


# Load the ontologies from server

In [2]:
Core_Ontology_URL="http://qptdat.plasma-mds.org/raw/core.ttl" 
Source_Ontology_URL="http://qptdat.plasma-mds.org/raw/source.ttl"

In [4]:
g = rdflib.Graph()
g.parse(Core_Ontology_URL)
g.parse(Source_Ontology_URL)

g.bind("qpt", rdflib.Namespace("http://qptdat.plasma-mds.org/ontology/"))
g.bind("qpt_source", rdflib.Namespace("http://qptdat.plasma-mds.org/source_ontology/"))
print(f"# triples :{len(g)}")

1609


Let's see which name spaces are already defined. These are important for SPARQL queries. 

In [5]:
for prefix, namespace in g.namespaces():
  print(prefix, namespace)

owl http://www.w3.org/2002/07/owl#
rdf http://www.w3.org/1999/02/22-rdf-syntax-ns#
rdfs http://www.w3.org/2000/01/rdf-schema#
xsd http://www.w3.org/2001/XMLSchema#
xml http://www.w3.org/XML/1998/namespace
dc http://purl.org/dc/elements/1.1/
obo http://purl.obolibrary.org/obo/
foaf http://xmlns.com/foaf/0.1/
obda https://w3id.org/obda/vocabulary#
orkg http://orkg.org/core#
qpkg http://qptdat.plasma-mds.org/resource/
p-plan http://purl.org/net/p-plan#
dcterms http://purl.org/dc/terms/
datacite http://purl.org/spar/datacite/
wikibase http://wikiba.se/ontology#
qpt http://qptdat.plasma-mds.org/ontology/
qpt_source http://qptdat.plasma-mds.org/source_ontology/


Now let's dive into the ontology and count the classes, properties, etc. 


In [31]:
query = """
SELECT (COUNT(DISTINCT ?class) AS ?class_count) (COUNT(DISTINCT ?object_property) AS ?object_property_count) 
        (COUNT(DISTINCT ?datatype_property) AS ?datatype_property_count) 
WHERE { 
    ?class a owl:Class .
    ?object_property a owl:ObjectProperty .
    ?datatype_property a owl:DatatypeProperty .
}
"""
qres = g.query(query)
for row in qres:
    print(f"classes {row.class_count}")
    print(f"properties {row.object_property_count + row.datatype_property_count}" 
          + f"\n\tobject properties {row.object_property_count}"
          + f"\n\tdatatype properties {row.datatype_property_count}")

classes 199
properties 57
	object properties 47
	datatype properties 10


In [30]:
query = """
SELECT (COUNT(DISTINCT ?named_individual) AS ?named_individual_count)
WHERE { 
    ?named_individual a owl:NamedIndividual .
}
"""
qres = g.query(query)
for row in qres:
    print(f"named individuals {row.named_individual_count}")

named individuals 34


Have a look into specic classes and related properties

In [11]:
query = """
SELECT ?s 
WHERE { 
    ?s rdfs:subClassOf qpt:Device .
}
"""
qres = g.query(query)
for row in qres:
    print(f"Device: {row.s}")

Device: http://qptdat.plasma-mds.org/ontology/AdjustedDevice
Device: http://qptdat.plasma-mds.org/ontology/DiagnosticDevice
Device: http://qptdat.plasma-mds.org/ontology/Source


with reasoner

In [12]:
print (len(g))
rdfs = owlrl.CombinedClosure.RDFS_OWLRL_Semantics(g, False, False, False)
rdfs.closure()
rdfs.flush_stored_triples()
print (len(g))

1609
7537


results

In [13]:
qres = g.query(query)
for row in qres:
    print(f"Device: {row.s}")

Device: http://qptdat.plasma-mds.org/ontology/AdjustedDevice
Device: http://qptdat.plasma-mds.org/ontology/DiagnosticDevice
Device: http://qptdat.plasma-mds.org/ontology/Source
Device: http://qptdat.plasma-mds.org/ontology/Profilometer
Device: http://www.w3.org/2002/07/owl#Nothing
Device: http://qptdat.plasma-mds.org/ontology/Detector
Device: http://qptdat.plasma-mds.org/ontology/Oscilloscope
Device: http://qptdat.plasma-mds.org/source_ontology/miller_auto_axcess_450
Device: http://qptdat.plasma-mds.org/ontology/Voltmeter
Device: http://qptdat.plasma-mds.org/ontology/MultipoleResonanceProbe
Device: http://qptdat.plasma-mds.org/ontology/LangmuirProbe
Device: http://qptdat.plasma-mds.org/source_ontology/minimip
Device: http://qptdat.plasma-mds.org/ontology/PlasmaJet
Device: http://qptdat.plasma-mds.org/ontology/DBD
Device: http://qptdat.plasma-mds.org/ontology/TransientRecorder
Device: http://qptdat.plasma-mds.org/ontology/AtomProbeTomographe
Device: http://qptdat.plasma-mds.org/ontology

So, wrapping things up until now ... 
There are two examples for modeling the cooking process.
We can query for Eggs in each of the examples. With reasoning active, we can also collect instances of sublcasses of Egg-classes. 

We now want to query for eggs in both examples, but with only one query. 
This will not work, because there is no mapping between the classes.

To include a mapping, we can load the PMDCO2 ontology. 

In [ ]:
#Mapping_URL="https://demo.fiz-karlsruhe.de/pmd/staticResources/data/pmdco_bfo2-mod.ttl"
Mapping_URL="https://demo.fiz-karlsruhe.de/pmd/staticResources/data/pmdco_bfo2-mod2.ttl"
g.parse(Mapping_URL)

<Graph identifier=Nb4920e7ca3594c0d8649b54eb6b6deef (<class 'rdflib.graph.Graph'>)>

We also have to include a part of the CCO ontology.

In [ ]:
CCO_URL="https://raw.githubusercontent.com/CommonCoreOntology/CommonCoreOntologies/master/ArtifactOntology.ttl"
g.parse(CCO_URL, format='ttl')

<Graph identifier=Nb4920e7ca3594c0d8649b54eb6b6deef (<class 'rdflib.graph.Graph'>)>

In [ ]:
BFO_URL="http://purl.obolibrary.org/obo/bfo/2020/bfo.owl"
g.parse(BFO_URL, format='xml')

<Graph identifier=Nb4920e7ca3594c0d8649b54eb6b6deef (<class 'rdflib.graph.Graph'>)>

Now lets do the reasoning (takes one or two minutes)

In [ ]:
print (len(g))
rdfs = owlrl.CombinedClosure.RDFS_OWLRL_Semantics(g, False, False, False)
rdfs.closure()
rdfs.flush_stored_triples()
print (len(g))

6850
43032


We now query again for **provo entities**, and the result is also showing entities from the cco example (these were not explicitly modelled as provo entities).

In [ ]:
query = """
SELECT ?s 
WHERE { 
    ?s a prov:Entity
}
"""
qres = g.query(query)
for row in qres:
    print(f"{row.s}")

http://example.org/provo#egg1
http://example.org/provo#cookingParams1
http://example.org/provo#easterEgg1
http://example.org/provo#boiledEgg1
http://example.org/ccobfo#Egg1
http://example.org/ccobfo#informationBearingEntity1
http://example.org/ccobfo#portionOfWater1
http://example.org/ccobfo#informationBearingEntity3
http://example.org/ccobfo#portionOfWater2
http://example.org/ccobfo#temperature2
http://example.org/ccobfo#temperature1
http://example.org/ccobfo#hardness1
http://example.org/ccobfo#eggSizeM
http://example.org/ccobfo#ShellColor
http://example.org/ccobfo#TapWater


Lets now add the other examples to the graph and start the reasoner again.

In [ ]:
g.parse(Egg_Ontology_EMMO_URL)
g.parse(Egg_Ontology_PMDCO_URL)
print (len(g))
rdfs = owlrl.CombinedClosure.RDFS_OWLRL_Semantics(g, False, False, False)
rdfs.closure()
rdfs.flush_stored_triples()
print (len(g))

43330
45095


Again the list of **provo entities**. Now with even more items.

In [ ]:
query = """
SELECT ?s 
WHERE { 
    ?s a prov:Entity
}
"""
qres = g.query(query)
for row in qres:
    print(f"{row.s}")

http://example.org/provo#egg1
http://example.org/provo#cookingParams1
http://example.org/provo#easterEgg1
http://example.org/provo#boiledEgg1
http://example.org/ccobfo#Egg1
http://example.org/ccobfo#informationBearingEntity1
http://example.org/ccobfo#portionOfWater1
http://example.org/ccobfo#informationBearingEntity3
http://example.org/ccobfo#portionOfWater2
http://example.org/ccobfo#temperature2
http://example.org/ccobfo#temperature1
http://example.org/ccobfo#hardness1
http://example.org/ccobfo#eggSizeM
http://example.org/ccobfo#ShellColor
http://example.org/ccobfo#TapWater
http://example.org/pmd#2e7012fc14764843a96b6e231fb67c53
http://example.org/pmd#515c392524f74d14bfd4e24c64141095
http://example.org/pmd#boiledEgg1
http://example.org/pmd#egg1
http://example.org/pmd#easterEgg1


Lets now show all **entities** and their **activities**!

The result comprises items from all examples.

In [ ]:
query = """
SELECT ?e ?p ?a 
WHERE { 
    ?e a prov:Entity .
    ?a a prov:Activity .
    ?e ?p ?a 
}
"""
qres = g.query(query)
for row in qres:
    print(f"{row.e}", f"{row.p}", f"{row.a}")

http://example.org/provo#boiledEgg1 http://www.w3.org/ns/prov#wasGeneratedBy http://example.org/provo#meCookingAnEgg1
http://example.org/provo#easterEgg1 http://www.w3.org/ns/prov#wasGeneratedBy http://example.org/provo#mePaintingAnEgg1
http://example.org/ccobfo#Egg1 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#PutEggInWater
http://example.org/ccobfo#portionOfWater1 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#PutEggInWater
http://example.org/ccobfo#Egg1 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#RetrieveEgg
http://example.org/ccobfo#Egg1 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#QuenchEgg
http://example.org/ccobfo#portionOfWater2 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#QuenchEgg
http://example.org/ccobfo#portionOfWater1 http://www.ontologyrepository

We are getting the same results, if we qeury for the **BFO process** (BFO_0000015) instead of **provo activity**

In [ ]:
query = """
SELECT ?e ?p ?a 
WHERE { 
    ?e a prov:Entity .
    ?a a obo:BFO_0000015 .
    ?e ?p ?a 
}
"""
qres = g.query(query)
for row in qres:
    print(f"{row.e}", f"{row.p}", f"{row.a}")

http://example.org/ccobfo#Egg1 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#QuenchEgg
http://example.org/ccobfo#portionOfWater2 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#QuenchEgg
http://example.org/ccobfo#Egg1 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#RetrieveEgg
http://example.org/ccobfo#Egg1 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#CookEgg
http://example.org/ccobfo#Egg1 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#PutEggInWater
http://example.org/ccobfo#portionOfWater1 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#PutEggInWater
http://example.org/ccobfo#portionOfWater1 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#CookWater
http://example.org/provo#boiledEgg1

Its also the same, if we query for the **BFO object**, instead of **provo entity**. 

In [ ]:
query = """
SELECT ?e ?p ?a 
WHERE { 
    ?e a obo:BFO_0000030 .
    ?a a obo:BFO_0000015 .
    ?e ?p ?a 
}
"""
qres = g.query(query)
for row in qres:
    print(f"{row.e}", f"{row.p}", f"{row.a}")

http://example.org/ccobfo#portionOfWater2 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#QuenchEgg
http://example.org/ccobfo#Egg1 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#QuenchEgg
http://example.org/ccobfo#Egg1 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#RetrieveEgg
http://example.org/ccobfo#Egg1 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#CookEgg
http://example.org/ccobfo#portionOfWater1 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#PutEggInWater
http://example.org/ccobfo#Egg1 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#PutEggInWater
http://example.org/ccobfo#portionOfWater1 http://www.ontologyrepository.com/CommonCoreOntologies/has_input http://example.org/ccobfo#CookWater
http://example.org/provo#boiledEgg1

In [ ]:
for prefix, namespace in g.namespaces():
  print(prefix, namespace)

rdf http://www.w3.org/1999/02/22-rdf-syntax-ns#
rdfs http://www.w3.org/2000/01/rdf-schema#
xsd http://www.w3.org/2001/XMLSchema#
xml http://www.w3.org/XML/1998/namespace
dc http://purl.org/dc/elements/1.1/
cco http://www.ontologyrepository.com/CommonCoreOntologies/
obo http://purl.obolibrary.org/obo/
ccobfo_egg http://example.org/ccobfo#
provo_egg http://example.org/provo#
prov http://www.w3.org/ns/prov#
err http://www.daml.org/2002/03/agents/agent-ont#
 https://material-digital.de/pmdco/
default1 http://www.ontologyrepository.com/CommonCoreOntologies/Mid/ArtifactOntology#
skos http://www.w3.org/2004/02/skos/core#
emmo_egg http://example.org/emmo#
owl http://www.w3.org/2002/07/owl#
pmdco http://material-digital.de/pmdco#
wikibase http://wikiba.se/ontology#
pmdao_egg http://example.org/pmd#


In [ ]:
query = """
SELECT ?e
WHERE { 
    #?e a <http://purl.obolibrary.org/obo/BFO_0000031> .   # bfo gen dep cont
    ?e a <http://emmo.info/emmo#EMMO_b21a56ed_f969_4612_a6ec_cb7766f7f31d> .   # emmo sign
   
    #?e a cco:InformationContentEntity .
}
"""
qres = g.query(query)
for row in qres:
    print(f"{row.e}")

http://example.org/ccobfo#ordinalMeasurementInformationContentEntity1
http://example.org/ccobfo#hardnessGoal
http://example.org/ccobfo#QuenchWaterTempSpec
http://www.ontologyrepository.com/CommonCoreOntologies/MinuteMeasurementUnit
http://example.org/ccobfo#intervalMeasurementInformationContentEntity1
http://example.org/ccobfo#eggSizeInitialm
http://example.org/ccobfo#PotWaterTempSpec
http://example.org/ccobfo#eggCookingPlan
http://example.org/ccobfo#eggCoockingTime1
http://www.ontologyrepository.com/CommonCoreOntologies/DegreeCelsiusMeasurementUnit
http://example.org/ccobfo#eggCookingPlanObjectiv
http://example.org/ccobfo#TapWaterName
http://example.org/emmo#HeatingWater1
http://example.org/emmo#EggBoiler1
http://example.org/emmo#EggBoiling1
http://example.org/emmo#BoiledEgg1
http://example.org/emmo#EggBoilingProcess1
http://example.org/emmo#Quenching1
http://example.org/emmo#Egg1
